# Facial Expression Recognition

## Import Essential Libraries

In [1]:
import os
import matplotlib.pyplot as plt
import shutil
import math
from PIL import Image
import cv2
import tensorflow as tf
import random
import pandas as pd
import seaborn as sns

2025-07-03 11:19:20.031185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751541560.315427      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751541560.400939      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Random Seed

In [2]:
# Set all random seeds (Python, NumPy, and TensorFlow)
tf.keras.utils.set_random_seed(42)

random.seed(42)

## Config Tensorflow to use GPU

In [3]:
import tensorflow as tf

# List all physical devices (CPUs and GPUs) that TensorFlow can see
physical_devices = tf.config.list_physical_devices()
print(f"Physical devices detected: {physical_devices}")

# Specifically list GPUs
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"\nNumber of GPUs available: {len(gpu_devices)}")
    for i, gpu in enumerate(gpu_devices):
        print(f"  GPU {i}: {gpu}")
    print("\nTensorFlow will automatically use the GPU if available.")
else:
    print("\nNo GPU devices found. TensorFlow will run on CPU.")

# You can also check if a random tensor is placed on GPU by default
# This should show GPU if one is available and being used
test_tensor = tf.constant([1.0, 2.0, 3.0])
print(f"\nDefault device for a tensor: {test_tensor.device}")

Physical devices detected: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

No GPU devices found. TensorFlow will run on CPU.

Default device for a tensor: /job:localhost/replica:0/task:0/device:CPU:0


2025-07-03 11:19:37.644262: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Global Variables

In [4]:
# List of emotions
emotions = ['angry', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [5]:
# List of emotion lables
emotion_labels = {
    0: 'anger',
    1: 'contempt',
    2: 'disgust',
    3: 'fear',
    4: 'happy',
    5: 'neutral',
    6: 'sad',
    7: 'surprise'
}

In [6]:
# Input related URLs
input_base_url = '/kaggle/input/'

train_images_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/train/images')
valid_images_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/valid/images')
test_images_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/test/images')

train_labels_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/train/labels')
valid_labels_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/valid/labels')
test_labels_url = os.path.join(input_base_url, 'affectnet-yolo-format/YOLO_format/test/labels')

In [12]:
# Output related URLs
output_base_url = '/kaggle/working/'

organized_train_images = os.path.join(output_base_url, 'organized_images/train')
organized_valid_images = os.path.join(output_base_url, 'organized_images/valid')
organized_test_images = os.path.join(output_base_url, 'organized_images/test')

## Split Images into Emotion Folders

In [11]:
def reorganize_dataset(source_images_dir, source_labels_dir, destination_base_dir, emotion_map):
    """
    Reorganizes image files into emotion-specific subfolders based on YOLO-format label files.

    Args:
        source_images_dir (str): Path to the directory containing image files (e.g., train/images).
        source_labels_dir (str): Path to the directory containing label .txt files (e.g., train/labels).
        destination_base_dir (str): Path to the root directory where reorganized data will be saved.
                                    (e.g., /kaggle/working/processed_train)
        emotion_map (dict): A dictionary mapping integer class IDs to emotion names (e.g., {0: 'anger'}).
    """
    print(f"--- Reorganizing: {source_images_dir.split('/')[-2]} set ---")

    # Create destination directories for each emotion
    for emotion_id, emotion_name in emotion_map.items():
        # Using the emotion name (e.g., 'anger') as the subfolder name
        class_folder = os.path.join(destination_base_dir, emotion_name)
        os.makedirs(class_folder, exist_ok=True) # exist_ok=True prevents error if folder already exists
        print(f"  Created directory: {class_folder}")

    # Iterate through each image file
    image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"  Found {len(image_files)} image files in {source_images_dir}")

    processed_count = 0
    skipped_count = 0

    for img_filename in image_files:
        img_path = os.path.join(source_images_dir, img_filename)
        
        # Construct the corresponding label file name
        # Remove the image extension (.png, .jpg) and replace with .txt
        base_filename = os.path.splitext(img_filename)[0]
        label_filename = base_filename + '.txt'
        label_path = os.path.join(source_labels_dir, label_filename)

        if not os.path.exists(label_path):
            print(f"    Warning: Label file not found for {img_filename} at {label_path}. Skipping.")
            skipped_count += 1
            continue

        try:
            with open(label_path, 'r') as f:
                # Read the first line (assuming one object/emotion per image)
                label_line = f.readline().strip()
                # Extract the class_id (first number)
                class_id = int(label_line.split(' ')[0])

            emotion_name = emotion_map.get(class_id)
            if emotion_name is None:
                print(f"    Warning: Unknown class_id {class_id} for {img_filename}. Skipping.")
                skipped_count += 1
                continue

            destination_folder = os.path.join(destination_base_dir, emotion_name)
            destination_path = os.path.join(destination_folder, img_filename)

            # Copy the image file
            shutil.copy(img_path, destination_path)
            processed_count += 1

            if processed_count % 1000 == 0:
                print(f"    Processed {processed_count} images for {source_images_dir.split('/')[-2]} set...")

        except Exception as e:
            print(f"    Error processing {img_filename} or {label_filename}: {e}. Skipping.")
            skipped_count += 1
            continue

    print(f"--- Finished reorganizing {source_images_dir.split('/')[-2]} set. Processed: {processed_count}, Skipped: {skipped_count} ---")
    print(f"Reorganized data is in: {destination_base_dir}\n")

In [14]:
os.makedirs(organized_train_images, exist_ok=True)

# Reorganize the training data
reorganize_dataset(train_images_url, train_labels_url, organized_train_images, emotion_labels)

--- Reorganizing: train set ---
  Created directory: /kaggle/working/organized_images/train/anger
  Created directory: /kaggle/working/organized_images/train/contempt
  Created directory: /kaggle/working/organized_images/train/disgust
  Created directory: /kaggle/working/organized_images/train/fear
  Created directory: /kaggle/working/organized_images/train/happy
  Created directory: /kaggle/working/organized_images/train/neutral
  Created directory: /kaggle/working/organized_images/train/sad
  Created directory: /kaggle/working/organized_images/train/surprise
  Found 17101 image files in /kaggle/input/affectnet-yolo-format/YOLO_format/train/images
    Processed 1000 images for train set...
    Processed 2000 images for train set...
    Processed 3000 images for train set...
    Processed 4000 images for train set...
    Processed 5000 images for train set...
    Processed 6000 images for train set...
    Processed 7000 images for train set...
    Processed 8000 images for train set...

In [15]:
os.makedirs(organized_valid_images, exist_ok=True)

# Reorganize the validation data
reorganize_dataset(valid_images_url, valid_labels_url, organized_valid_images, emotion_labels)

--- Reorganizing: valid set ---
  Created directory: /kaggle/working/organized_images/valid/anger
  Created directory: /kaggle/working/organized_images/valid/contempt
  Created directory: /kaggle/working/organized_images/valid/disgust
  Created directory: /kaggle/working/organized_images/valid/fear
  Created directory: /kaggle/working/organized_images/valid/happy
  Created directory: /kaggle/working/organized_images/valid/neutral
  Created directory: /kaggle/working/organized_images/valid/sad
  Created directory: /kaggle/working/organized_images/valid/surprise
  Found 5406 image files in /kaggle/input/affectnet-yolo-format/YOLO_format/valid/images
    Processed 1000 images for valid set...
    Processed 2000 images for valid set...
    Processed 3000 images for valid set...
    Processed 4000 images for valid set...
    Processed 5000 images for valid set...
--- Finished reorganizing valid set. Processed: 5406, Skipped: 0 ---
Reorganized data is in: /kaggle/working/organized_images/val

In [16]:
os.makedirs(organized_test_images, exist_ok=True)

# Reorganize the validation data
reorganize_dataset(test_images_url, test_labels_url, organized_test_images, emotion_labels)

--- Reorganizing: test set ---
  Created directory: /kaggle/working/organized_images/test/anger
  Created directory: /kaggle/working/organized_images/test/contempt
  Created directory: /kaggle/working/organized_images/test/disgust
  Created directory: /kaggle/working/organized_images/test/fear
  Created directory: /kaggle/working/organized_images/test/happy
  Created directory: /kaggle/working/organized_images/test/neutral
  Created directory: /kaggle/working/organized_images/test/sad
  Created directory: /kaggle/working/organized_images/test/surprise
  Found 2755 image files in /kaggle/input/affectnet-yolo-format/YOLO_format/test/images
    Processed 1000 images for test set...
    Processed 2000 images for test set...
--- Finished reorganizing test set. Processed: 2755, Skipped: 0 ---
Reorganized data is in: /kaggle/working/organized_images/test

